In [1]:
#sys path
from sys import path
from pathlib import Path

module_path = str(Path.cwd().parents[0])

if module_path not in path:
    path.append(module_path)
    
path.append(module_path + '\\functions')


# libraries
import pandas as pd
import numpy as np

import scipy.linalg
import matplotlib.pyplot as plt

from gp_regression import GPR
from gp_ccopf_hybrid import GP_CCOPF
import save_outputs
import linear_model
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from sklearn.metrics import mean_squared_error

# Dataset

In [2]:
# Data Set AC

df_ac = pd.read_csv('datasets\dataset_pandapower_AC.csv')
df_ac = df_ac.drop(['Unnamed: 0'], axis=1)
df_ac[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
   'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28']] = df_ac[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
   'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28']]*345
df_ac.head()

,Pg0,Pg1,Pg2,Pg3,Pg4,Pg5,Pg6,Pg7,Pg8,Pg9,Load_P1,Load_P2,Load_P3,Load_P4,Load_P5,Load_P6,Load_P7,Load_P8,Load_P9,Load_P10,Load_P11,Load_P12,Load_P13,Load_P14,Load_P15,Load_P16,Load_P17,Load_P18,Load_P19,Load_P20,Load_P21,Load_Q1,Load_Q2,Load_Q3,Load_Q4,Load_Q5,Load_Q6,Load_Q7,Load_Q8,Load_Q9,Load_Q10,Load_Q11,Load_Q12,Load_Q13,Load_Q14,Load_Q15,Load_Q16,Load_Q17,Load_Q18,Load_Q19,Load_Q20,Load_Q21,RS_P1,RS_P2,RS_P3,RS_P4,RS_P5,RS_P6,RS_Q1,RS_Q2,RS_Q3,RS_Q4,RS_Q5,RS_Q6,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,U0,U1,U2,U3,U4,U5,U6,U7,U8,U9,U10,U11,U12,U13,U14,U15,U16,U17,U18,U19,U20,U21,U22,U23,U24,U25,U26,U27,U28,P_0_1,P_0_38,P_1_2,P_1_24,P_2_3,P_2_17,P_3_4,P_3_13,P_4_5,P_4_7,P_5_6,P_5_10,P_6_7,P_7_8,P_8_38,P_9_10,P_9_12,P_12_13,P_13_14,P_14_15,P_15_16,P_15_18,P_15_20,P_15_23,P_16_17,P_16_26,P_20_21,P_21_22,P_22_23,P_22_35,P_24_25,P_25_26,P_25_27,P_25_28,P_27_28
0,543.683861,480.780962,422.589181,515.193702,465.726434,398.126985,407.492184,438.353382,470.090281,474.440983,90.914192,278.257547,484.890806,191.941725,434.717963,5.701385,7.723471,280.393552,294.915708,144.022588,619.300718,248.616210,237.842081,292.766489,207.294727,122.702763,244.951684,195.564458,269.750389,7.756971,1067.487076,41.172206,2.073969,178.439817,68.961099,147.071250,-58.417271,79.679422,134.063167,28.953731,27.346061,93.805844,104.346220,81.298748,-87.469444,43.679960,15.006813,65.814420,26.201840,25.595363,3.878486,241.731675,201.939244,186.743432,171.992000,155.919662,253.889380,180.249926,67.313081,62.247811,57.330667,51.973221,84.629793,60.083309,-97.245241,-125.249374,-87.828262,-26.192747,-121.630562,-63.966173,-14.113978,44.594445,119.362797,132.576888,357.533796,366.516282,362.426592,354.176395,353.704441,354.278588,351.097342,350.706460,359.376335,358.097584,356.796693,358.532398,358.048084,357.867344,362.180939,365.939434,365.604158,364.022800,366.401697,344.593204,364.795027,368.511407,366.799821,369.030342,367.683643,370.339039,366.668991,369.656607,369.003772,-446.072007,355.157815,353.544169,-123.383733,135.603873,-61.800089,-51.929770,-297.719713,-453.478557,401.526746,474.814290,-396.796605,281.551854,246.800549,239.617760,300.379826,122.209355,203.098223,-95.668618,-204.266790,288.185103,-355.725130,-235.824246,-196.241467,206.146955,81.521045,-484.849331,67.504248,236.381820,-406.757379,105.405929,163.890125,-67.209717,-114.323150,-262.947709
1,536.484387,547.951193,477.059316,470.671430,569.117091,440.880892,548.190410,603.776581,650.451261,598.786405,108.212196,366.011893,599.722807,243.260219,596.746247,7.403969,10.074933,373.254003,377.338547,188.271444,748.880865,268.522571,280.726052,340.867608,238.451631,157.474964,305.898926,223.975171,312.723399,10.007058,1209.309080,49.005933,2.728039,220.697993,87.398881,201.887715,-75.862208,103.938349,178.462070,37.045699,35.747742,113.433425,112.701079,95.957269,-101.840549,50.245165,19.259528,82.189925,30.008324,29.672873,5.003529,273.847165,374.974885,216.516176,209.977880,328.995585,202.538118,237.571862,124.991628,72.172059,69.992627,109.665195,67.512706,79.190621,-155.705930,-196.957456,-165.711546,-70.372389,-163.112166,-102.193941,-59.125669,6.617990,71.855892,65.285740,354.271657,363.317417,356.261718,346.412652,346.743619,347.704588,343.437462,342.949162,357.424456,352.669139,350.852239,351.455937,352.134207,351.065401,355.405436,360.536256,359.720738,357.802904,364.026889,342.294748,360.489328,366.785366,364.050651,363.684132,365.823557,365.389146,360.489024,366.166932,366.753763,-535.056085,426.843889,602.339887,-224.014487,277.785890,-45.866380,11.228432,-334.308754,-468.752007,479.978902,558.565468,-488.789839,313.386853,194.351398,185.732984,374.961711,102.097605,192.078070,-143.485948,-307.155271,221.121345,-284.799691,-400.004073,-221.754352,234.541353,-13.734384,-669.707577,96.846686,362.881792,-546.846719,135.478053,320.999974,-145.009345,-198.511163,-369.796724
2,474.376160,519.312942,435.221241,543.465098,471.367214,584.793955,444.998575,616.119147,425.730377,596.9259

In [3]:
# Data Set DC

df_dc = pd.read_csv('datasets\dataset_pandapower_DC.csv')
df_dc = df_dc.drop(['Unnamed: 0'], axis=1)
df_dc[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
   'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28']] = df_dc[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
   'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28']]*345
df_dc.head()

,Pg0,Pg1,Pg2,Pg3,Pg4,Pg5,Pg6,Pg7,Pg8,Pg9,Load_P1,Load_P2,Load_P3,Load_P4,Load_P5,Load_P6,Load_P7,Load_P8,Load_P9,Load_P10,Load_P11,Load_P12,Load_P13,Load_P14,Load_P15,Load_P16,Load_P17,Load_P18,Load_P19,Load_P20,Load_P21,Load_Q1,Load_Q2,Load_Q3,Load_Q4,Load_Q5,Load_Q6,Load_Q7,Load_Q8,Load_Q9,Load_Q10,Load_Q11,Load_Q12,Load_Q13,Load_Q14,Load_Q15,Load_Q16,Load_Q17,Load_Q18,Load_Q19,Load_Q20,Load_Q21,RS_P1,RS_P2,RS_P3,RS_P4,RS_P5,RS_P6,RS_Q1,RS_Q2,RS_Q3,RS_Q4,RS_Q5,RS_Q6,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,U0,U1,U2,U3,U4,U5,U6,U7,U8,U9,U10,U11,U12,U13,U14,U15,U16,U17,U18,U19,U20,U21,U22,U23,U24,U25,U26,U27,U28,P_0_1,P_0_38,P_1_2,P_1_24,P_2_3,P_2_17,P_3_4,P_3_13,P_4_5,P_4_7,P_5_6,P_5_10,P_6_7,P_7_8,P_8_38,P_9_10,P_9_12,P_12_13,P_13_14,P_14_15,P_15_16,P_15_18,P_15_20,P_15_23,P_16_17,P_16_26,P_20_21,P_21_22,P_22_23,P_22_35,P_24_25,P_25_26,P_25_27,P_25_28,P_27_28
0,543.683861,480.780962,422.589181,515.193702,465.726434,398.126985,407.492184,438.353382,470.090281,474.440983,90.914192,278.257547,484.890806,191.941725,434.717963,5.701385,7.723471,280.393552,294.915708,144.022588,619.300718,248.616210,237.842081,292.766489,207.294727,122.702763,244.951684,195.564458,269.750389,7.756971,1067.487076,41.172206,2.073969,178.439817,68.961099,147.071250,-58.417271,79.679422,134.063167,28.953731,27.346061,93.805844,104.346220,81.298748,-87.469444,43.679960,15.006813,65.814420,26.201840,25.595363,3.878486,241.731675,201.939244,186.743432,171.992000,155.919662,253.889380,180.249926,67.313081,62.247811,57.330667,51.973221,84.629793,60.083309,-140.0,-169.245894,-150.0,-57.794674,-124.011569,-44.607391,-73.82297,-1.322610e-08,116.848744,131.941981,353.533139,363.78227,358.029858,349.524611,349.235369,350.058282,345.938268,345.422171,355.43417,354.871346,353.184938,354.835178,354.437705,353.375994,356.304144,359.764934,359.89262,358.723235,361.287426,338.985924,357.754339,361.950265,361.006237,362.57024,365.7,365.652696,361.104576,364.958137,364.558577,-448.074817,357.160625,365.968853,-131.323464,148.404073,-60.692767,-37.017108,-299.469625,-435.662973,398.645864,469.600677,-409.035853,277.658952,241.586853,235.885468,309.842460,112.746722,192.573289,-106.896336,-215.297888,287.609252,-361.619418,-239.140056,-197.063374,204.715355,82.893896,-487.756266,66.290380,235.940484,-407.492184,99.735191,162.057788,-68.888493,-116.136866,-264.452951
1,536.484387,547.951193,477.059316,470.671430,569.117091,440.880892,548.190410,603.776581,650.451261,598.786405,108.212196,366.011893,599.722807,243.260219,596.746247,7.403969,10.074933,373.254003,377.338547,188.271444,748.880865,268.522571,280.726052,340.867608,238.451631,157.474964,305.898926,223.975171,312.723399,10.007058,1209.309080,49.005933,2.728039,220.697993,87.398881,201.887715,-75.862208,103.938349,178.462070,37.045699,35.747742,113.433425,112.701079,95.957269,-101.840549,50.245165,19.259528,82.189925,30.008324,29.672873,5.003529,273.847165,374.974885,216.516176,209.977880,328.995585,202.538118,237.571862,124.991628,72.172059,69.992627,109.665195,67.512706,79.190621,-140.0,-169.245894,-150.0,-57.794674,-124.011569,-44.607391,-73.82297,-1.322610e-08,116.848744,131.941981,353.533139,363.78227,358.029858,349.524611,349.235369,350.058282,345.938268,345.422171,355.43417,354.871346,353.184938,354.835178,354.437705,353.375994,356.304144,359.764934,359.89262,358.723235,361.287426,338.985924,357.754339,361.950265,361.006237,362.57024,365.7,365.652696,361.104576,364.958137,364.558577,-539.369592,431.157396,623.474888,-239.918401,302.075403,-44.612408,38.876323,-336.523727,-437.630426,476.506749,550.268964,-508.231949,307.008746,186.769248,179.365279,390.116274,86.943042,175.268610,-161.255118,-324.531241,219.526642,-290.907656,-406.182110,-224.306665,232.883852,-13.357210,-674.704680,95.171797,362.636155,-548.190410,125.406548,319.256136,-148.606075,-202.718477,-372.581246
2,474.376160,519.312942,435.221241,543.465098,471.367214,584.793955,444.998575,616.119147,425.730377,596.925906,99.077568,343.352262,500.9

In [4]:
df = df_ac - df_dc
df.head()

,Pg0,Pg1,Pg2,Pg3,Pg4,Pg5,Pg6,Pg7,Pg8,Pg9,Load_P1,Load_P2,Load_P3,Load_P4,Load_P5,Load_P6,Load_P7,Load_P8,Load_P9,Load_P10,Load_P11,Load_P12,Load_P13,Load_P14,Load_P15,Load_P16,Load_P17,Load_P18,Load_P19,Load_P20,Load_P21,Load_Q1,Load_Q2,Load_Q3,Load_Q4,Load_Q5,Load_Q6,Load_Q7,Load_Q8,Load_Q9,Load_Q10,Load_Q11,Load_Q12,Load_Q13,Load_Q14,Load_Q15,Load_Q16,Load_Q17,Load_Q18,Load_Q19,Load_Q20,Load_Q21,RS_P1,RS_P2,RS_P3,RS_P4,RS_P5,RS_P6,RS_Q1,RS_Q2,RS_Q3,RS_Q4,RS_Q5,RS_Q6,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,U0,U1,U2,U3,U4,U5,U6,U7,U8,U9,U10,U11,U12,U13,U14,U15,U16,U17,U18,U19,U20,U21,U22,U23,U24,U25,U26,U27,U28,P_0_1,P_0_38,P_1_2,P_1_24,P_2_3,P_2_17,P_3_4,P_3_13,P_4_5,P_4_7,P_5_6,P_5_10,P_6_7,P_7_8,P_8_38,P_9_10,P_9_12,P_12_13,P_13_14,P_14_15,P_15_16,P_15_18,P_15_20,P_15_23,P_16_17,P_16_26,P_20_21,P_21_22,P_22_23,P_22_35,P_24_25,P_25_26,P_25_27,P_25_28,P_27_28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.754759,43.996521,62.171738,31.601927,2.381007,-19.358782,59.708992,44.594445,2.514053,0.634907,4.000657,2.734012,4.396734,4.651784,4.469071,4.220306,5.159073,5.284289,3.942166,3.226239,3.611755,3.697220,3.610379,4.491350,5.876794,6.174500,5.711538,5.299566,5.114272,5.607281,7.040687,6.561142,5.793584,6.460102,1.983643,4.686343,5.564415,4.698470,4.445195,2.002810,-2.002810,-12.424684,7.939730,-12.800200,-1.107322,-14.912662,1.749912,-17.815585,2.880881,5.213613,12.239248,3.892902,5.213695,3.732292,-9.462633,9.462633,10.524934,11.227718,11.031098,0.575852,5.894288,3.315810,0.821907,1.431600,-1.372852,2.906935,1.213867,0.441336,0.734806,5.670738,1.832337,1.678776,1.813716,1.505243
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.705930,-27.711561,-15.711546,-12.577715,-39.100597,-57.586550,14.697300,6.617990,-44.992852,-66.656241,0.738518,-0.464853,-1.768140,-3.111959,-2.491750,-2.353695,-2.500807,-2.473009,1.990286,-2.202207,-2.332700,-3.379241,-2.303498,-2.310593,-0.898708,0.771322,-0.171882,-0.920331,2.739463,3.308824,2.734988,4.835101,3.044414,1.113892,0.123557,-0.263550,-0.615552,1.208795,2.195186,4.313507,-4.313507,-21.135001,15.903914,-24.289513,-1.253972,-27.647892,2.214974,-31.121581,3.472153,8.296504,19.442110,6.378108,7.582150,6.367705,-15.154563,15.154563,16.809461,17.769170,17.375971,1.594704,6.107964,6.178037,2.552313,1.657501,-0.377174,4.997103,1.674889,0.245637,1.343690,10.071505,1.743837,3.596731,4.207314,2.784523
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.687173,9.025244,13.871286,7.489049,-14.768082,-53.665192,38.322889,17.340381,3.382386,-32.402445,2.066209,0.290908,0.837674,0.588324,0.302783,0.193363,0.289168,0.474305,3.084355,-0.297011,-0.319796,-3.276017,-0.152702,0.879090,3.746854,3.506789,2.664941,1.978018,3.885252,4.556680,4.350462,5.490454,4.192292,3.513809,1.080857,2.685475,2.566251,4.041530,4.456258,3.567562,-3.567562,-17.981975,13.575246,-18.879237,-1.830930,-22.515288,2.792678,-25.737548,3.183752,6.886699,16.356612,5.191804,6.401760,5.328495,-12.719643,12.719643,14.276879,16.106551,15.032275,0.014344,6.100461,6.475398,1.943359,2.410920,-3.261745,5.234178,1.789611,0.731918,0.880817,7.882417,3.928281,1.442132,1.703975,1.316656
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [5]:
def _get_x(df):
    X = df[['Pg0', 'Pg1', 'Pg2', 'Pg3', 'Pg4', 'Pg5', 'Pg6', 'Pg7', 'Pg8', 'Pg9',
            'Load_P1', 'Load_P2', 'Load_P3', 'Load_P4', 'Load_P5', 'Load_P6','Load_P7', 'Load_P8', 'Load_P9',
            'Load_P10', 'Load_P11', 'Load_P12', 'Load_P13', 'Load_P14', 'Load_P15','Load_P16', 'Load_P17', 'Load_P18',
            'Load_P19', 'Load_P20', 'Load_P21', 'RS_P1', 'RS_P2', 'RS_P3', 'RS_P4', 'RS_P5', 'RS_P6']]
    return X 


def _get_y(df):
    y = df[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
            'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28',
            'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38',
            'P_0_1', 'P_0_38', 'P_1_2', 'P_1_24', 'P_2_3', 'P_2_17', 'P_3_4', 'P_3_13', 'P_4_5', 'P_4_7', 'P_5_6', 'P_5_10', 'P_6_7',
            'P_7_8', 'P_8_38', 'P_9_10', 'P_9_12', 'P_12_13', 'P_13_14', 'P_14_15', 'P_15_16', 'P_15_18', 'P_15_20', 'P_15_23', 'P_16_17', 'P_16_26',
            'P_20_21', 'P_21_22','P_22_23', 'P_22_35', 'P_24_25', 'P_25_26', 'P_25_27', 'P_25_28', 'P_27_28']]
    return y 


X = _get_x(df_ac)
Y = _get_y(df)

In [6]:
Nu = 10
Nd = 27

n_train = 200
n_test = n_train + 65

X_train = X.iloc[:n_train]  
Y_train = Y.iloc[:n_train]
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()


X_test = X.iloc[n_train:n_test]
Y_test = Y.iloc[n_train:n_test]
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

# GP - Train and Save model

In [7]:
model = 0

M=10
Nc=1000

print('*****************')
print('GP')
print('*****************')

#Train
if model==0:
    gp = GPR(X_train, Y_train, M, Nc, normalize=True)
    gp.save_model('models/IEEE39_hybrid')

#Load    
elif model==1:
    gp = GPR.load_model('models/IEEE39_hybrid')


print('*****************')
print('Hyper_parameters')
print('*****************')
gp.print_hyper_parameters()

*****************
GP
*****************

________________________________________
# Optimizing hyperparameters (N=200)
----------------------------------------
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1795090656855.6926
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 1:  0.259444 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1806705202497.316
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 2:  0.233810 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1796662390812.3164
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 3:  0.271749 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1800207920876.3057
            Iterations: 1
            Fun

Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1772528603493.977
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 36:  0.224402 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1784439324654.121
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 37:  0.262585 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1766572488960.8403
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 38:  0.262704 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1801217222065.354
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 39:  0.274904 s
Inequality constraints incompatible    (Exit mode 4)
            Current fu

Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1775544763717.483
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 72:  0.234968 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1778836692152.0527
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 73:  0.225497 s
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 1747230398772.9253
            Iterations: 1
            Function evaluations: 40
            Gradient evaluations: 1
* Output 74:  0.312850 s
----------------------------------------
*****************
Hyper_parameters
*****************

________________________________________
# Hyper-parameters
----------------------------------------
* Num samples: 200
* Ny: 74
* Nx: 37
* Normalization: True
----------------------------------------
* Lengths

# GP - Validation

In [8]:
print('*****************')
print('Validation - Validation Data')
print('*****************')

RMSE = gp.validate(X_test, Y_test, case='IEEE39')
print('RMSE:', np.mean(RMSE))

*****************
Validation - Validation Data
*****************

________________________________________
# Validation of GP model 
----------------------------------------
* Num training samples: 200
* Num test samples: 65
----------------------------------------
* Root Mean Squared Error: 
	- State 1: 0.006102
	- State 2: 0.007049
	- State 3: 0.009860
	- State 4: 0.012126
	- State 5: 0.011261
	- State 6: 0.010257
	- State 7: 0.012147
	- State 8: 0.012175
	- State 9: 0.005159
	- State 10: 0.009367
	- State 11: 0.010137
	- State 12: 0.014455
	- State 13: 0.010230
	- State 14: 0.011357
	- State 15: 0.012452
	- State 16: 0.010331
	- State 17: 0.010319
	- State 18: 0.010392
	- State 19: 0.005171
	- State 20: 0.004463
	- State 21: 0.009524
	- State 22: 0.006122
	- State 23: 0.006445
	- State 24: 0.010269
	- State 25: 0.004851
	- State 26: 0.007989
	- State 27: 0.009974
	- State 28: 0.006402
	- State 29: 0.005152
	- State 30: 0.111903
	- State 31: 0.130548
	- State 32: 0.130587
	- State 33

# GP CCOPF hybrid

### Linear Model Coefficients

In [9]:
save = 1

if save==0:
    dc = pd.read_csv('datasets\dataset_pandapower_DC.csv')
    A_coef, b_inter = linear_model.linear_model_IEEE39(dc)
    
    IEEE_dict = save_outputs.to_dict_coef(A_coef, b_inter)
    save_outputs.save_model(IEEE_dict, 'outputs\linear_coeff')
    
elif save==1:
    
    coeff_dict = save_outputs.load_model('outputs\linear_coeff')
    
    A_coef = np.array(coeff_dict['A'])
    b_inter = np.array(coeff_dict['b'])

### Input parameters

In [10]:
# Input mean
Xd = np.array([[ 97.6,  322.,  500., 233.8,   522.,   6.5,   8.53,   320.,  329., 158.,  680.,   274.,   247.5,  308.6,
                 224.,  139.,  281.,  206.,  283.5,   9.2,  1104., 
                 210., 210., 210., 210.,  210.,  210. ]])

# Participation factors
alpha = np.array([[0.117, 0.117, 0.117, 0.117, 0.0178, 0.1173, 0.0883, 0.0726, 0.117, 0.117]]).T

# Costs
R = np.eye(Nu) * [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
R2 = np.ones(Nu) * [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

# Control boundaries
ulb = [   0,    0,   0,   0,   0,   0,   0,   0,   0,    0]
uub = [ 646, 1040, 725, 652, 508, 687, 580, 564, 865, 1100]

# Output boundaries
xlb = [310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5,
       310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5, 310.5,
       310.5, 
      -500,  -500.,  -500.,  -500.,  -500.,  -500.,   -500.,  -500.,  -500., -500.,
      -2000, -2000,  -2000,  -2000,  -2000,  -2000,   -2000,  -2000,  -2000, -2000,  -2000,  -2000,  -2000,  -2000,  
      -2000, -2000,  -2000,  -2000,  -2000,  -2000,   -2000,  -2000,  -2000, -2000,  -2000,  -2000,  -2000,  -2000,
      -2000, -2000,  -2000,  -2000,  -2000,  -2000,   -2000
           ]
           
xub = [379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 
       379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5, 379.5,
       379.5,
       500,    500.,   500.,   500.,   500.,   500.,    500.,   500.,   500.,  500.,
       2000,   2000,   2000,   2000,   2000,   2000,    2000,   2000,   2000,  2000,   2000,   2000,   2000,   2000,  
       2000,   2000,   2000,   2000,   2000,   2000,    2000,   2000,   2000,  2000,   2000,   2000,   2000,   2000,
       2000,   2000,   2000,   2000,   2000,   2000,    2000
          ]

# Percentages
per_load = 0.15 
per_rs   = 0.3 
n_load   = 21

# DC
dc_u = list(df_dc[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
                   'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28',
                   'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38']].iloc[0])

### TA1

In [11]:
gp_opf_TA1 = GP_CCOPF(gp=gp, gp_method='TA1', predefine_probability_y=0.975, predefine_probability_u=0.999,
                      R=R, R2=R2, ulb=ulb, uub=uub, xlb=xlb, xub=xub, solver_opts=None, normalize=True,
                      X=X_train, Y=Y_train, Xd=Xd, alpha=alpha, per_load=per_load, per_rs=per_rs, n_load=n_load, A_coef=A_coef, b_inter=b_inter, dc_u=dc_u,
                      system='IEEE39')             

----------------------------------------
# Time to build GP CCOPF solver: 0.197515 sec


In [12]:
u_TA1, alpha_TA1, x_TA1, s_TA1, trace_covar_TA1, Xd_std = gp_opf_TA1.solve()

# Save outputs
gp_save = 0

if gp_save == 0:
    IEEE_dict = save_outputs.to_dict(x_TA1, s_TA1, u_TA1, alpha_TA1, trace_covar_TA1)
    save_outputs.save_model(IEEE_dict, 'outputs\IEEE39_TA1_hybrid')


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Total number of variables............................:      168
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      150
Total number of inequality constraints...............:      252
        inequality constraints with only lower bounds:      168
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       84


Number of Iterations....: 11

    

# Error metric

In [16]:
gp = GPR.load_model('models/IEEE39_hybrid')


TA1_dict = save_outputs.load_model('outputs\IEEE39_TA1_hybrid')
x_TA1 = np.array(TA1_dict['x'])
s_TA1 = np.array(TA1_dict['s'])
u_TA1 = np.array(TA1_dict['u'])
alpha_TA1 = np.array(TA1_dict['s'])
u_TA1 = np.array(TA1_dict['trace_covar'])

In [17]:
# Data Set
df_pf_ac_mean = pd.read_csv('datasets\dataset_tested_real_AC.csv')
df_pf_ac_mean[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
   'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28']] = df_pf_ac_mean[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16',
   'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28']]*345
df_pf_ac_mean.head()

def _get_outputs_ac(df_ac):
    V_ac = df_ac[['U0', 'U1','U2', 'U3', 'U4', 'U5', 'U6','U7', 'U8','U9', 'U10', 'U11', 'U12', 'U13', 'U14', 'U15', 'U16', 
             'U17', 'U18','U19', 'U20', 'U21', 'U22', 'U23','U24', 'U25','U26', 'U27', 'U28',
             'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34','Q35', 'Q36', 'Q37', 'Q38',
             'P_0_1', 'P_0_38', 'P_1_2', 'P_1_24', 'P_2_3', 'P_2_17', 'P_3_4', 'P_3_13', 'P_4_5', 'P_4_7', 'P_5_6', 'P_5_10', 'P_6_7',
             'P_7_8', 'P_8_38', 'P_9_10', 'P_9_12', 'P_12_13', 'P_13_14', 'P_14_15', 'P_15_16', 'P_15_18', 'P_15_20', 'P_15_23', 'P_16_17', 'P_16_26',
             'P_20_21', 'P_21_22','P_22_23', 'P_22_35', 'P_24_25', 'P_25_26', 'P_25_27', 'P_25_28', 'P_27_28']]
    return V_ac 

out_pf_ac_mean = _get_outputs_ac(df_pf_ac_mean)

out_pf_ac_mean = np.array(out_pf_ac_mean)
out_pf_ac_meanout_pf_ac_mean = out_pf_ac_mean[:39, :]
out_pf_ac_mean.shape

(1, 74)

In [15]:
out_pf_mean_pu = np.zeros((1,74))
x_TA1_pu = np.zeros((1,74))

out_pf_mean_pu[:, :29] = out_pf_ac_mean[:, :29]/345
out_pf_mean_pu[:, 29:] = out_pf_ac_mean[:, 29:]/400

x_TA1_pu[:, :29] = x_TA1[:, :29]/345
x_TA1_pu[:, 29:] = x_TA1[:, 29:]/400



RMSE_average = mean_squared_error(out_pf_mean_pu.T, x_TA1_pu.T, multioutput='uniform_average', squared=False)
RMSE_row = mean_squared_error(out_pf_mean_pu, x_TA1_pu, multioutput='raw_values', squared=False)

print('RMSE_avrage:', RMSE_average)
print('')
print('RMSE_row:', RMSE_row)

RMSE_avrage: 0.014106061284549732

RMSE_row: [0.00017854 0.00214667 0.00324065 0.00324615 0.00255605 0.00147865
 0.00238391 0.00225488 0.00032574 0.00286636 0.00290128 0.00568957
 0.00331314 0.00372202 0.00548844 0.00450355 0.00407617 0.0037714
 0.00207781 0.00154369 0.00390159 0.00180511 0.00219502 0.0048629
 0.00223013 0.0033518  0.00383476 0.0029618  0.00280198 0.04137267
 0.03244277 0.03776013 0.03623602 0.02641167 0.0424215  0.0367706
 0.03163315 0.04717535 0.00618078 0.00864099 0.0052319  0.00789539
 0.00343698 0.00335493 0.00276799 0.0055113  0.00267806 0.0156071
 0.01055852 0.01241788 0.01874814 0.01008566 0.02029287 0.00159626
 0.00544383 0.00424739 0.00354825 0.00187761 0.00161436 0.00639781
 0.00478684 0.00033277 0.0013493  0.00308668 0.00445746 0.00061424
 0.00173704 0.00013008 0.00214736 0.00125777 0.00353411 0.00141945
 0.00187561 0.00276741]
